In [1]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")
review = ["This film was a waste of time","This film was very nice"]
tokenizer = classifier.tokenizer
result = tokenizer(review, padding = True, return_tensors = "pt")
print(result)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


{'input_ids': tensor([[ 101, 2023, 2143, 2001, 1037, 5949, 1997, 2051,  102],
        [ 101, 2023, 2143, 2001, 2200, 3835,  102,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]])}


In [11]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")
reviews = ["This film was a waste of time","This film was very nice"]
tokenizer = classifier.tokenizer
result = tokenizer(review, padding = True, return_tensors = "pt")
#print(result)
print(result['input_ids'])
print(result['attention_mask'])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


tensor([[ 101, 2023, 2143, 2001, 1037, 5949, 1997, 2051,  102],
        [ 101, 2023, 2143, 2001, 2200, 3835,  102,    0,    0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [3]:
classifier.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [7]:
model=classifier.model
outputs=model(result['input_ids'],result['attention_mask'])
print(outputs)


SequenceClassifierOutput(loss=None, logits=tensor([[ 4.7709, -3.8290],
        [-4.2027,  4.5809]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [9]:
import torch
probability_dist = torch.nn.functional.softmax(outputs.logits, dim=1)
print(probability_dist)

tensor([[9.9982e-01, 1.8409e-04],
        [1.5321e-04, 9.9985e-01]], grad_fn=<SoftmaxBackward0>)


In [12]:
class_labels = ["Negative", "Positive"]

# Convert predictions into human-readable format
for i, review in enumerate(reviews):
  negative_score = probability_dist[i, 0].item()
  positive_score = probability_dist[i, 1].item()
  predicted_label = class_labels[positive_score > negative_score]  # Choose label based on max score

  print(f"Review: \"{review}\"")
  print(f"Predicted Sentiment: {predicted_label} (Positive: {positive_score:.4f}, Negative: {negative_score:.4f})\n")

Review: "This film was a waste of time"
Predicted Sentiment: Negative (Positive: 0.0002, Negative: 0.9998)

Review: "This film was very nice"
Predicted Sentiment: Positive (Positive: 0.9998, Negative: 0.0002)



#code Explain

In [23]:
# i = 0 #i=1
# negative_score = probability_dist[i, 0].item()  # Gets the value 0.99982 as a Python float
# positive_score = probability_dist[i, 1].item()  # Gets the value 0.00018 as a Python float

# print("Negative score:", negative_score)
# print("Positive score:", positive_score)


Negative score: 0.9998158812522888
Positive score: 0.0001840865588746965


In [24]:
# cls = ["Negative", "Positive"]


# pos=0.2
# neg=0.8
# predicted_label = cls[pos > neg]
# print(predicted_label)

Negative


#AutoTokenizer

What is an attention mask?
An attention mask is a list (or tensor) of the same length as the input tokens that tells the model which tokens to pay attention to and which ones to ignore (usually padding tokens).

Why do we need it?
Handling variable-length inputs:
Models process batches of sentences simultaneously. But sentences have different lengths, so shorter sentences are padded with special [PAD] tokens to match the longest sentence length.

Ignoring padding tokens:
The padding tokens are not real words and contain no useful information. Without a mask, the model might consider them during attention, confusing the prediction.

Efficient computation:
By masking out padding tokens, the model’s attention layers skip these positions, improving performance and accuracy.

How does the attention mask work in your example?
python

'input_ids': [101, 7592, 2026, 2171, 2003, 2033, 2232, 11263, 2480, 102]
'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


All tokens are actual content (no padding), so the mask is all 1s, meaning pay attention to all tokens.

If you had a padded sentence, the mask might look like [1, 1, 1, 1, 0, 0] — the 0s tell the model to ignore those padding tokens.

Summary:
Attention masks help models know which tokens are real and which are padding.

They prevent the model from treating padding as meaningful input.

This improves model accuracy and efficiency, especially with batches of varying-length sentences.

In [29]:
from transformers import pipeline

import torch
sentiment_pipeline = pipeline(
task="sentiment-analysis",
model="distilbert-base-uncased-finetuned-sst-2-english"
)




tokenizer = sentiment_pipeline.tokenizer
print(tokenizer("Hello my name is Mehfuz"))

tokens=tokenizer.tokenize("Hello my name is Mehfuz")
print(tokens)


tokens_id=tokenizer.encode("Hello my name is Mehfuz")
print(tokens_id)

#using id we can get back our tokens string/actual input
decode_tokens=tokenizer.decode(tokens_id)
print(decode_tokens)

Device set to use cpu


{'input_ids': [101, 7592, 2026, 2171, 2003, 2033, 2232, 11263, 2480, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['hello', 'my', 'name', 'is', 'me', '##h', '##fu', '##z']
[101, 7592, 2026, 2171, 2003, 2033, 2232, 11263, 2480, 102]
[CLS] hello my name is mehfuz [SEP]


vocab = tokenizer.get_vocab()
print(f"Vocabulary size: {len(vocab)}")
If it prints, say, 30522, it means the tokenizer knows 30,522 unique tokens, and the model’s embedding layer expects token IDs in the range [0, 30521].

Vocabulary is basically a lookup table that maps tokens (words or subword pieces) to their unique token IDs.

When you give input text, the tokenizer uses this vocab to convert each token into its corresponding ID.

These token IDs are then fed into the model.

The vocab is specific to the tokenizer/model, so it knows exactly which tokens it recognizes and their IDs.

So yes — vocab = lookup table for converting input text into token IDs based on what the model knows.

In [30]:
tokenizer.get_vocab()

{'captures': 19566,
 'mor': 22822,
 'boring': 11771,
 'embassy': 8408,
 'dissatisfied': 25956,
 'linden': 22066,
 'fis': 27424,
 '[unused389]': 394,
 'tex': 16060,
 'conspicuous': 19194,
 'doha': 26528,
 'politician': 3761,
 'rumbled': 22257,
 'knife': 5442,
 'conscription': 26329,
 '[unused860]': 865,
 '##gel': 12439,
 'floors': 8158,
 'cancel': 17542,
 'baroness': 21479,
 '##itaire': 29422,
 'tanzania': 11959,
 '##ductive': 26638,
 'farewell': 13407,
 'artificial': 7976,
 'sponge': 25742,
 'protein': 5250,
 '##rna': 12789,
 'nils': 27282,
 'publishing': 4640,
 '##·': 29661,
 '##ver': 6299,
 'allegheny': 21192,
 'assam': 15181,
 'diocesan': 18680,
 '##rana': 16737,
 'synthetic': 12553,
 'cheering': 24867,
 '##thic': 23048,
 '62': 5786,
 '\\': 1032,
 'boeing': 10321,
 '[unused400]': 405,
 'clarkson': 18648,
 'heck': 17752,
 'dal': 17488,
 '##bia': 11607,
 'cotton': 6557,
 'drills': 28308,
 'governor': 3099,
 'lev': 23310,
 'simple': 3722,
 'mentor': 10779,
 'direct': 3622,
 'gameplay':

In [31]:
tokenizer.vocab_size

30522

In [32]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(tokenizer("Hello my name is Mehfuz"))

{'input_ids': [101, 7592, 2026, 2171, 2003, 2033, 2232, 11263, 2480, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


AutoModelForSequenceClassification is a Hugging Face model class designed for classification tasks, and yes, it outputs logits.

In [34]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare input text
text = "Hello my name is Mehfuz"

# Tokenize input and convert to tensors
inputs = tokenizer(text, return_tensors="pt")

# Forward pass through model
outputs = model(**inputs)

print(outputs)


SequenceClassifierOutput(loss=None, logits=tensor([[-0.4583,  0.7277]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


#Full Code

In [35]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# List of movie reviews for sentiment analysis
reviews = ["This film was a waste of time", "This film was very nice"]

# Extract the tokenizer from the pipeline
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the input reviews and convert them to tensors
tokenized_inputs = tokenizer(reviews, padding=True, return_tensors="pt")

# Extract the model from the pipeline
sentiment_model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Perform model inference using tokenized inputs
model_outputs = sentiment_model(
input_ids=tokenized_inputs["input_ids"],
attention_mask=tokenized_inputs["attention_mask"]
)

# Apply softmax to convert logits into probabilities
predicted_probabilities = torch.nn.functional.softmax(model_outputs.logits,
dim=1)

# Define labels for the classes (Index 0 = Negative, Index 1 = Positive)
class_labels = ["Negative", "Positive"]

# Convert predictions into human-readable format
for i, review in enumerate(reviews):
  negative_score = predicted_probabilities[i, 0].item()
  positive_score = predicted_probabilities[i, 1].item()
  # Determine predicted label based on highest probability
  predicted_label = class_labels[0] if negative_score > positive_score else class_labels[1]
  print(f"Review: \"{review}\"")
  print(f"Predicted Sentiment: {predicted_label} (Positive: {positive_score:.4f}, Negative: {negative_score:.4f})\n")

Review: "This film was a waste of time"
Predicted Sentiment: Negative (Positive: 0.0002, Negative: 0.9998)

Review: "This film was very nice"
Predicted Sentiment: Positive (Positive: 0.9998, Negative: 0.0002)

